In [1]:
import datetime
import os
import numpy as np
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
init_notebook_mode(connected=True)

from obs_data import *
from plotlyFunctions import *

print(os.getcwd())

2016
/Users/DayTightChunks/Documents/PhD/Models/phd-model-master/Analysis


### Path and working computer

In [2]:
PC = False
analysis = False
short_sim = True
version = "mFix_v1"
folder = "1"
timestamp = 'julian'

In [3]:
# Path to simulated folder
if analysis:
    dot = '.'
else:
    dot = '..'

# Working computer
if PC:
    back = "\\"  
    sim_path = version + back  
else:
    back = "/"
    sim_path = dot + back + version + back    
    
print(sim_path)

../mFix_v1/


In [4]:
# Function to add column ID, with first letter capitalized.
def newlabel(row, plot):
    return plot.capitalize() + '-' + str(int(row['Jdays']))

In [5]:
# TODO: Change t -> v (also in the observed data: 0.2_XYcalibrationSoils.Rmd)
detail_plots = ['n1', 'n2', 'n3', 'n4', 'n5', 'n7', 'n8',
                'v4', 'v5', 'v7', 'v8', 'v9', 
                's11', 's12', 's13']

sample_comp = ['nor', 'val', 'sou']

## Concentrations

In [6]:
variable = 'CONC'
name_obs = 'conc_det_cal'
matches = []
for sample in range(len(detail_plots)):
    plot = detail_plots[sample]
    filename = "resM_" + plot + variable + ".tss"
    det_sim = pd.read_table(sim_path + folder + back + filename, 
                                skiprows=4, delim_whitespace=True,
                                names=['Jdays', 'SIM'],
                                header=None)
    # Add ID
    det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, plot),axis=1) 
    n = getSoilDataCal(name_obs, tss=True)
    match = pd.merge(n, det_sim, how='inner', on='IDcal')
    matches.append(match)

# Concatenate detailed
conc_det_dat = pd.concat(matches)

# Prepare composites and concatenate
variable = 'CONC'
name_obs = 'conc_comp_cal'
matches = []
for sample in range(len(sample_comp)):
    plot = sample_comp[sample]
    filename = "resM_" + plot + variable + ".tss"
    det_sim = pd.read_table(sim_path + folder + back + filename, 
                                skiprows=4, delim_whitespace=True,
                                names=['Jdays', 'SIM'],
                                header=None)
    # Add ID (remove after running model)
    if sample == 1:
        label = "T"
    else:
        label = plot[0]
        
    det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, label),axis=1) 
    n = getSoilDataCal(name_obs, tss=True)
    match = pd.merge(n, det_sim, how='inner', on='IDcal')
    matches.append(match)
    
# Concatenate composites
conc_comp_dat = pd.concat(matches)

conc_dat = pd.concat([conc_det_dat, conc_comp_dat])

In [7]:
conc_dat.head()

,IDcal,Jdays_x,Jdays_y,SIM,Transect,ug.g,ug.g.SD
0,N1-181,181,181,0.05728,N,0.071209,NaN
1,N1-286,286,286,4.69364,N,4.289449,NaN
0,N2-181,181,181,7.14343,N,4.710027,NaN
1,N2-225,225,225,2.62046,N,4.781827,NaN
2,N2-272,272,272,1.26034,N,0.776233,NaN


## Delta

In [8]:
variable = 'd13C'
name_obs = 'delta_det_cal'
matches = []
for sample in range(len(detail_plots)):
    plot = detail_plots[sample]
    filename = "resM_" + plot + variable + ".tss"
    det_sim = pd.read_table(sim_path + folder + back + filename, 
                                skiprows=4, delim_whitespace=True,
                                names=['Jdays', 'SIM'],
                                header=None)
    # Add ID
    det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, plot),axis=1) 
    n = getSoilDataCal(name_obs, tss=True)
    match = pd.merge(n, det_sim, how='inner', on='IDcal')
    matches.append(match)
    
delta_det_dat = pd.concat(matches)

variable = 'd13C'
name_obs = 'delta_comp_cal'
matches = []
for sample in range(len(sample_comp)):
    plot = sample_comp[sample]
    filename = "resM_" + plot + variable + ".tss"
    det_sim = pd.read_table(sim_path + folder + back + filename, 
                                skiprows=4, delim_whitespace=True,
                                names=['Jdays', 'SIM'],
                                header=None)
    # Add ID (remove after running model)
    if sample == 1:
        label = "T"
    else:
        label = plot[0]
        
    det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, label),axis=1) 
    n = getSoilDataCal(name_obs, tss=True)
    match = pd.merge(n, det_sim, how='inner', on='IDcal')
    matches.append(match)
    
delta_comp_dat = pd.concat(matches)

delta_dat = pd.concat([delta_det_dat, delta_comp_dat])

In [9]:
delta_dat.tail()

,Jdays_x,Transect,IDcal,d13C,d13C.SD,Jdays_y,SIM
6,230,S,S-230,-29.466667,0.642910,230,-31.3669
7,244,S,S-244,-28.633333,0.288675,244,-31.2651
8,251,S,S-251,-28.933333,0.152753,251,-31.2282
9,258,S,S-258,-27.600000,0.435890,258,-31.1982
10,265,S,S-265,-28.166667,0.378594,265,-31.1740


## Plotting

### Concentrations

In [10]:
conc_dat.head()

,IDcal,Jdays_x,Jdays_y,SIM,Transect,ug.g,ug.g.SD
0,N1-181,181,181,0.05728,N,0.071209,NaN
1,N1-286,286,286,4.69364,N,4.289449,NaN
0,N2-181,181,181,7.14343,N,4.710027,NaN
1,N2-225,225,225,2.62046,N,4.781827,NaN
2,N2-272,272,272,1.26034,N,0.776233,NaN


In [11]:
test_var = 'ug.g'
sd = 'ug.g.SD'

data = [get_xy_line(conc_dat, test_var), 
        get_scatter(conc_dat, test_var, sd)]

fig = go.Figure(data=data, layout=get_layout(conc_dat, test_var, title='Conc. Soils (Sim vs. Obs)'))
iplot(fig)

### Isotopes

In [12]:
delta_dat.tail()

,Jdays_x,Transect,IDcal,d13C,d13C.SD,Jdays_y,SIM
6,230,S,S-230,-29.466667,0.642910,230,-31.3669
7,244,S,S-244,-28.633333,0.288675,244,-31.2651
8,251,S,S-251,-28.933333,0.152753,251,-31.2282
9,258,S,S-258,-27.600000,0.435890,258,-31.1982
10,265,S,S-265,-28.166667,0.378594,265,-31.1740


In [13]:
test_var = 'd13C'
sd = 'd13C.SD'

data = [get_xy_line(delta_dat, test_var, delta=True), 
        get_scatter(delta_dat, test_var, sd)]

fig = go.Figure(data=data, layout=get_layout(delta_dat, test_var, title='Delta Soils (Sim vs. Obs)'))
iplot(fig)

In [14]:
df = delta_dat.loc[delta_dat['Transect'] == 'N']

data = [get_xy_line(df , test_var, delta=True), 
        get_scatter(df , test_var, sd)]

fig = go.Figure(data=data, layout=get_layout(df, test_var, title='North - Delta Soils (Sim vs. Obs)'))
iplot(fig)

In [15]:
df = delta_dat.loc[delta_dat['Transect'] == 'T']

data = [get_xy_line(df , test_var, delta=True), 
        get_scatter(df , test_var, sd)]

fig = go.Figure(data=data, layout=get_layout(df, test_var, title='Valley - Delta Soils (Sim vs. Obs)'))
iplot(fig)

In [16]:
df = delta_dat.loc[delta_dat['Transect'] == 'S']

data = [get_xy_line(df , test_var, delta=True), 
        get_scatter(df , test_var, sd)]

fig = go.Figure(data=data, layout=get_layout(df, test_var, title='South - Delta Soils (Sim vs. Obs)'))
iplot(fig)